In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import time
start_time = time.time()
import pandas as pd
import numpy as np
import spacy
# from tensorflow import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

data = pd.read_csv("../input/train.csv",header=None,low_memory=False)
data_test = pd.read_csv("../input/test.csv",header=None,low_memory=False)
sentences = data[1][1:]
sentences_test = data_test[1][1:]
X = sentences.values
Y = data[2][1:]
Y = Y.values.astype(int)
nlp = spacy.load('en_core_web_lg')
nlp.remove_pipe('ner')
nlp.remove_pipe('parser')
nlp.remove_pipe('tagger')
nlp.max_length = 93621305
print("{:.3f} seconds taken".format(time.time() - start_time))

In [ ]:
start_time = time.time()
Y_train = Y
sentence_to_vec = []
for doc in nlp.pipe(X, batch_size=250, n_threads=4):
    sentence_to_vec.append(doc.vector)
print("{:.3f} seconds taken".format(time.time() - start_time))

In [ ]:
start_time = time.time()
LR_model = LogisticRegression(solver='lbfgs', max_iter=2000).fit(sentence_to_vec, Y_train)
print("{:.3f} seconds taken".format(time.time() - start_time))

In [ ]:
start_time = time.time()
X_test = []
for doc in nlp.pipe(sentences_test, batch_size=250, n_threads=4):
    X_test.append(doc.vector)
print("{:.3f} seconds taken".format(time.time() - start_time))

In [ ]:
start_time = time.time()
test_preds = LR_model.predict_proba(X_test)[:,1]
print("{:.3f} seconds taken".format(time.time() - start_time))
print(len(test_preds), test_preds[:10])

In [ ]:
train_preds = LR_model.predict_proba(sentence_to_vec)[:,1]
y_train = Y_train

def bestThresshold(y_train,train_preds):
    # since we know that the function in concave
    bd = (0.0, 0.5, 1.0) # left, middle, right
    val = (0.0, f1_score(y_train, np.array(train_preds)>bd[1]), 0.0)
    for _ in range(20):
        left_mid = 0.5*(bd[0]+bd[1])
        right_mid = 0.5*(bd[1]+bd[2])
        left_mid_val = f1_score(y_train, np.array(train_preds)>left_mid)
        right_mid_val = f1_score(y_train, np.array(train_preds)>right_mid)
        if left_mid_val > right_mid_val:
            bd = (bd[0], left_mid, bd[1])
            print('when threshold is {:.6f}, F1 score is {:.6f}'.format(bd[1], left_mid_val))
        else:
            bd = (bd[1], right_mid, bd[2])
            print('when threshold is {:.6f}, F1 score is {:.6f}'.format(bd[1], right_mid_val))
    return bd[1]

delta = bestThresshold(Y_train,train_preds)

In [ ]:
sub = pd.read_csv("../input/sample_submission.csv")
sub["prediction"] = (test_preds > delta).astype(int)
sub.to_csv("submission.csv",index=False)